# Map Design Studio Demo using a FastAPI backend

The porpose of this demo is to show how to use the Map Design Studio to create a map.

## Setup

### Library import


In [1]:
# imports
import os

import folium
import numpy as np
import requests
from dotenv import load_dotenv
from folium.plugins import VectorGridProtobuf
from matplotlib.colors import LinearSegmentedColormap
from pydantic import BaseModel

# Load the environment variables from the .env file
load_dotenv()

True

**Constants**

In [2]:
# Constants
API_BASE_URL = "http://127.0.0.1:8088"

### Utils

In [3]:
def rgb_to_hex(rgb):
    """
    Convert RGB color values to hexadecimal representation.

    Args:
        rgb (tuple): A tuple of three integers representing the RGB color values.

    Returns:
        str: The hexadecimal representation of the RGB color values.

    Example:
        >>> rgb_to_hex((255, 0, 0))
        '#ff0000'
    """
    return "#{:02x}{:02x}{:02x}".format(*rgb)


def generate_styles_dict(colors):
    """
    Generate a dictionary of color map values from a list of colors.

    Args:
        colors (list): A list of colors.

    Returns:
        dict: A dictionary of color map values.

    """
    cmap = LinearSegmentedColormap.from_list("cmap", colors, 256)
    x = np.linspace(0, 1, 256)
    cmap_vals = cmap(x)[:, :]
    cmap_uint8 = (cmap_vals * 255).astype("uint8")
    cmap_dict = {str(idx): rgb_to_hex(tuple(value)[:-1]) for idx, value in enumerate(cmap_uint8)}
    return cmap_dict

In [4]:
class RasterData(BaseModel):
    """
    Represents the data of a raster layer.
    """

    id: int
    styles: dict


class VectorData(BaseModel):
    """
    Represents the data of a vector layer.
    """

    id: int

<a id='section_1'></a>
## Demo

### Create Layers
#### Continuous raster layer
**Styles**

In [5]:
colors = [
    '#422112',
    '#724C01',
    '#CEA712',
    '#FFA904',
    '#FDFE00',
    '#E6EC06',
    '#BACF00',
    '#8BB001',
    '#72A002',
    '#5B8D03',
    '#448102',
    '#2C7001',
    '#176100',
]
styles_dict = generate_styles_dict(colors)

**Get Tiles**

In [6]:
raster_data = RasterData(id=1, styles=styles_dict)  # noqa: F821
response_raster_continuous = requests.get(
    f"{API_BASE_URL}/raster_tiles", json=raster_data.model_dump()
)

raster_continuous = response_raster_continuous.json()

#### Categorical raster layer
**Styles**

In [5]:
styles_dict = {
    "0": "#1327FE",
    "1": "#0B4614",
    "2": "#0C691B",
    "3": "#54A51B",
    "4": "#76D01E",
    "5": "#DCCF5C",
    "6": "#B4FE25",
    "7": "#DADC4D",
    "8": "#C25045",
    "9": "#A4A4A4",
    "10": "#64FFF8",
    "11": "#F9FEA4",
}

**Get Tiles**

In [6]:
raster_data = RasterData(id=2, styles=styles_dict)  # noqa: F821
response_raster_categorical = requests.get(
    f"{API_BASE_URL}/raster_tiles", json=raster_data.model_dump()
)

raster_categorical = response_raster_categorical.json()

#### Vector layer

**Get Tiles**

In [8]:
vector_data = VectorData(id=3)
response_vector = requests.get(f"{API_BASE_URL}/vector_tiles", json=vector_data.model_dump())

vector = response_vector.json()

### Create Map

#### Basemap

For a list of basemaps providers go to https://leaflet-extras.github.io/leaflet-providers/preview/.
You can also provide a custom basemap:

In [11]:
# Get the value of the MAPBOX_API_KEY environment variable
mapbox_api_key = os.getenv("MAPBOX_API_KEY")

In [12]:
custom_basemap = (
    "https://api.mapbox.com/styles/v1/lce1/clocw2j2d016501qx90go7rwt/tiles/256/{z}/{x}/{y}@2x?"
    + f"access_token={mapbox_api_key}"
)
attr = (
    '© <a href="https://www.mapbox.com/about/maps/">Mapbox</a> '
    '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> '
    '<strong><a href="https://www.mapbox.com/map-feedback/" '
    'target="_blank">Improve this map</a></strong>'
)

#### Map

In [13]:
m = folium.Map(tiles=custom_basemap, attr=attr)

**Add raster layers**

In [14]:
# Add continuou raster layer to the map
tile_layer = folium.TileLayer(
    tiles=raster_continuous.get("url"),
    name=raster_continuous.get("info").get("name"),
    attr=" ",
    overlay=True,
    control=True,
    opacity=1,
)

tile_layer.add_to(m)

# Add categorical raster layer to the map
tile_layer = folium.TileLayer(
    tiles=raster_categorical.get("url"),
    name=raster_categorical.get("info").get("name"),
    attr=" ",
    overlay=True,
    control=True,
    opacity=1,
)

tile_layer.add_to(m)

**Add vector layers**

In [15]:
styles_dict = {
    "fill": True,
    "weight": 1,
    "fillColor": "#06cccc",
    "color": "#06cccc",
    "fillOpacity": 0.1,
    "opacity": 1,
}

layer_name = vector.get("layer_name")

vector_tile_layer_styles = {}
vector_tile_layer_styles[layer_name] = styles_dict
options = {"vectorTileLayerStyles": vector_tile_layer_styles}

In [16]:
tile_layer = VectorGridProtobuf(
    url=vector.get("url"),
    name=vector.get("info").get("name"),
    options=options,
    overlay=True,
    control=True,
)

tile_layer.add_to(m)

**Add control to turn on/off the layer**

In [17]:
# Add control to turn on/off the layer
control = folium.LayerControl(position="topright")
control.add_to(m)
m.show_in_browser()